In [1]:
import numpy as np
import pandas as pd
import contractions

# Import nltk package 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

# scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

# for regular expression
import re

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/Jackie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Jackie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import os
current_directory = os.getcwd()
desktop_directory = os.path.join(os.path.expanduser("~"), "Desktop")
file_name = "designcodata.csv"
file_path = os.path.join(desktop_directory, file_name)

DCO_df = pd.read_csv(file_path)


In [4]:
DCO_df.head(5)

,Timestamp,Participant's Name:,What options for social interaction do you have using this screen?,"What would you expect to happen next, after tapping an icon in “connect with friends”?",What would you like to see embedded in the “Weekly challenges” section?,What would you like to see embedded in the “explore local events” section?,How can this page improve? What changes or additions would you make to the current Home page?,"How effective is this page in supporting your sustainability journey on this app?\n(More on scale: the middle is somewhat useful in its current state, needs improvement)",Explain your rating,Interact with the Home Page to navigate to your Profile Page.,...,Did you encounter any features that were confusing or not helpful to your interaction with this app/ overall sustainability journey?,Can you think of anything that was missing in this wireframe that you were hoping to find in a sustainability app?,"On a scale of 1-10, how likely are you to use an app developed from this wireframe to meet your sustainability needs and goals?\n(More on Scale: the middle would be I would use a sustainability app, but if it were different than what I saw in this wireframe.)",Explain your rating.5,Any last thoughts or lingering questions about this wireframe?,"Consider this alternate Learn Page, what are some features or design choices that stand out to you about this page?",What can you infer this page is about? How did you come to that conclusion?,How would you search up videos to learn more about a sustainability habit?,"How can this page improve? What are some interactions or features you would hope to see included here? (And, why?)","Considering this alternate Learn Page, do you prefer this page or the original Learn page? What can the final Learn Page implement from these versions?"
0,2/27/2024 22:31:42,Valeria Palomares,I can click on my friends and probably message...,It would show me what they achievements they’v...,Maybe a daily challenge for every day of the week,Event images with descriptions of things to do...,some more color i guess this looks intersting ...,7,it makes sense to have a homepage I don't know...,Completed with ease,...,No not really. The layers were a bit confusing...,I guess it didn’t have the profile edit page l...,10,I think it is packed with a lot of resources a...,"I really like this one, maybe change the way l...",I can see this is a videos page because it loo...,This is a page to learn about sustainable habi...,I think you enter your search where it says “w...,A way to filter video content,i like the way the other page lets you learn s...
1,3/3/2024 13:26:52,Aariana Lopez,Connect with people w similar interests,Add ppl make new friends,In a week how many plastic bottles can you fil...,"Probably like garage sales, basically swap mee...","Colors, water bottle background, earthy colors...",7,"It’s missing colors to attract people, and min...",Completed with ease,...,"No, i didn’t see anything that was not helpful...",Just the actual games it would be more fun and...,8,It’s good to have an app to involve yourself m...,I think it would be fun and I am looking forwa...,That there is a whole page dedicated to the vi...,It’s a learn page,I would use the search bar at the top,It could have options for likeing or reactions...,I like the other page but since I did say it n...
2,3/3/2024 21:04:00,Alice,"Connecting with friends, doing weekly challeng...",Either bring me to their profiles page or a di...,I would like to see a progress bar shows how m...,"Information about the date, time, and location...",I feel like the navigation icons are kind of v...,7,I think it could benefit from a section that r...,Completed with ease,...,The layers feature was confusing to use and I ...,No,7,I think the goal behind this app is great and ...,What's the most central goal of this app? Is i...,"It is pretty straightforward, which I like",I can infer that it will have videos covering ...,I would search up videos based o

For the HomePage Screen we want to answer the following question for all 7 participants that completed the user testing.
The first question we want to answer: Do users find this screen useful? 
However, first let's explore the data found for the HomePage screen by doing some text analysis.

In [5]:
# lets check for null values in the dataframe
check_null = DCO_df.isnull().sum()
print(check_null.sum())
# there should be no null values

# lets also drop the timestamp column first because it really is not necessary
column_to_drop = 'Timestamp'
DCO_df.drop(column_to_drop, axis=1, inplace=True)
# check that the column was dropped

0


In [6]:
DCO_df.head(5)

,Participant's Name:,What options for social interaction do you have using this screen?,"What would you expect to happen next, after tapping an icon in “connect with friends”?",What would you like to see embedded in the “Weekly challenges” section?,What would you like to see embedded in the “explore local events” section?,How can this page improve? What changes or additions would you make to the current Home page?,"How effective is this page in supporting your sustainability journey on this app?\n(More on scale: the middle is somewhat useful in its current state, needs improvement)",Explain your rating,Interact with the Home Page to navigate to your Profile Page.,What information can learn about your account on this page?,...,Did you encounter any features that were confusing or not helpful to your interaction with this app/ overall sustainability journey?,Can you think of anything that was missing in this wireframe that you were hoping to find in a sustainability app?,"On a scale of 1-10, how likely are you to use an app developed from this wireframe to meet your sustainability needs and goals?\n(More on Scale: the middle would be I would use a sustainability app, but if it were different than what I saw in this wireframe.)",Explain your rating.5,Any last thoughts or lingering questions about this wireframe?,"Consider this alternate Learn Page, what are some features or design choices that stand out to you about this page?",What can you infer this page is about? How did you come to that conclusion?,How would you search up videos to learn more about a sustainability habit?,"How can this page improve? What are some interactions or features you would hope to see included here? (And, why?)","Considering this alternate Learn Page, do you prefer this page or the original Learn page? What can the final Learn Page implement from these versions?"
0,Valeria Palomares,I can click on my friends and probably message...,It would show me what they achievements they’v...,Maybe a daily challenge for every day of the week,Event images with descriptions of things to do...,some more color i guess this looks intersting ...,7,it makes sense to have a homepage I don't know...,Completed with ease,i'm not sure what that means because you can't...,...,No not really. The layers were a bit confusing...,I guess it didn’t have the profile edit page l...,10,I think it is packed with a lot of resources a...,"I really like this one, maybe change the way l...",I can see this is a videos page because it loo...,This is a page to learn about sustainable habi...,I think you enter your search where it says “w...,A way to filter video content,i like the way the other page lets you learn s...
1,Aariana Lopez,Connect with people w similar interests,Add ppl make new friends,In a week how many plastic bottles can you fil...,"Probably like garage sales, basically swap mee...","Colors, water bottle background, earthy colors...",7,"It’s missing colors to attract people, and min...",Completed with ease,"It shows when I joined, how many friends, some...",...,"No, i didn’t see anything that was not helpful...",Just the actual games it would be more fun and...,8,It’s good to have an app to involve yourself m...,I think it would be fun and I am looking forwa...,That there is a whole page dedicated to the vi...,It’s a learn page,I would use the search bar at the top,It could have options for likeing or reactions...,I like the other page but since I did say it n...
2,Alice,"Connecting with friends, doing weekly challeng...",Either bring me to their profiles page or a di...,I would like to see a progress bar shows how m...,"Information about the date, time, and location...",I feel like the navigation icons are kind of v...,7,I think it could benefit from a section that r...,Completed with ease,When I joined the app\nhow many friends I have...,...,The layers feature was confusing to use and I ...,No,7,I think the goal behind this app is great and ...,What's the most central goal o

In [7]:
# anonymize the participant names to make identification more standardized and keep names confidential
# replace the names for stand in identifiers user 1, user 2, etc.
replace_names = {'Valeria Palomares':'user1', 'Aariana Lopez ':'user2', 'Alice':'user3', 'Kelly La':'user4', 'Stephanie ':'user5', 'Yinqi Li':'user6', 'Kate ':'user7'}
DCO_df["Participant's Name:"] = DCO_df["Participant's Name:"].replace(replace_names)
# check to see if values changed
print (DCO_df["Participant's Name:"])
# some names did not convert, make sure to add a space after the name

0    user1
1    user2
2    user3
3    user4
4    user5
5    user6
6    user7
Name: Participant's Name:, dtype: object


In [8]:
# convert all the responses to lowercase
def preprocess_text(x):
    """
    Preprocess text in a DataFrame cell by converting to lowercase,
    removing punctuation, and separating words with spaces.

    Parameters:
    - x: cell value 
    - this func gets applied to a df

    Returns:
    - Preprocessed cell value
    """
    # initialize result_string variable
    result_string = x
    
    if isinstance(x, str):
        # Convert to lowercase
        x = x.lower()
        # Use contractions library to handle contractions
        x = contractions.fix(x)
        # Remove punctuation and separate words with spaces
        x = re.sub(r'[^A-Za-z0-9]+', ' ', x)
        # use NLTK to handle contraction words
        words = word_tokenize(x) 
        result_string = ' '.join(words)
        
    return result_string

In [9]:
# apply this function to every cell in DCO_df
DCO_df = DCO_df.applymap(preprocess_text)
# check df
DCO_df.head(1)

,Participant's Name:,What options for social interaction do you have using this screen?,"What would you expect to happen next, after tapping an icon in “connect with friends”?",What would you like to see embedded in the “Weekly challenges” section?,What would you like to see embedded in the “explore local events” section?,How can this page improve? What changes or additions would you make to the current Home page?,"How effective is this page in supporting your sustainability journey on this app?\n(More on scale: the middle is somewhat useful in its current state, needs improvement)",Explain your rating,Interact with the Home Page to navigate to your Profile Page.,What information can learn about your account on this page?,...,Did you encounter any features that were confusing or not helpful to your interaction with this app/ overall sustainability journey?,Can you think of anything that was missing in this wireframe that you were hoping to find in a sustainability app?,"On a scale of 1-10, how likely are you to use an app developed from this wireframe to meet your sustainability needs and goals?\n(More on Scale: the middle would be I would use a sustainability app, but if it were different than what I saw in this wireframe.)",Explain your rating.5,Any last thoughts or lingering questions about this wireframe?,"Consider this alternate Learn Page, what are some features or design choices that stand out to you about this page?",What can you infer this page is about? How did you come to that conclusion?,How would you search up videos to learn more about a sustainability habit?,"How can this page improve? What are some interactions or features you would hope to see included here? (And, why?)","Considering this alternate Learn Page, do you prefer this page or the original Learn page? What can the final Learn Page implement from these versions?"
0,user1,i can click on my friends and probably message...,it would show me what they achievements they h...,maybe a daily challenge for every day of the week,event images with descriptions of things to do...,some more color i guess this looks intersting ...,7,it makes sense to have a homepage i do not kno...,completed with ease,i am not sure what that means because you can ...,...,no not really the layers were a bit confusing ...,i guess it did not have the profile edit page ...,10,i think it is packed with a lot of resources a...,i really like this one maybe change the way lo...,i can see this is a videos page because it loo...,this is a page to learn about sustainable habi...,i think you enter your search where it says wa...,a way to filter video content,i like the way the other page let us you learn...


In [10]:
# inspect a specific cell in row 1 further to see if there will be a problem with contractions
DCO_df.iat[0,9]

'i am not sure what that means because you can not learn much you can only see my achievements and sustainable locations but i do not know what sustainable locations means so i do not know what i would learn from that i do not think the achievements makes sense either since they are my personal achievements unless it means like awards that i have i guess you could learn how many friends i have and a streak but a streak is more personal incentive so you can not learn much there is no bio i do not know'

In [11]:
# add a Page Type to help split the questions in df
page_type = 'HomePage'
DCO_df.insert(0, 'PageType', page_type)

In [12]:
# check the column was added successfully
DCO_df.head(2)

,PageType,Participant's Name:,What options for social interaction do you have using this screen?,"What would you expect to happen next, after tapping an icon in “connect with friends”?",What would you like to see embedded in the “Weekly challenges” section?,What would you like to see embedded in the “explore local events” section?,How can this page improve? What changes or additions would you make to the current Home page?,"How effective is this page in supporting your sustainability journey on this app?\n(More on scale: the middle is somewhat useful in its current state, needs improvement)",Explain your rating,Interact with the Home Page to navigate to your Profile Page.,...,Did you encounter any features that were confusing or not helpful to your interaction with this app/ overall sustainability journey?,Can you think of anything that was missing in this wireframe that you were hoping to find in a sustainability app?,"On a scale of 1-10, how likely are you to use an app developed from this wireframe to meet your sustainability needs and goals?\n(More on Scale: the middle would be I would use a sustainability app, but if it were different than what I saw in this wireframe.)",Explain your rating.5,Any last thoughts or lingering questions about this wireframe?,"Consider this alternate Learn Page, what are some features or design choices that stand out to you about this page?",What can you infer this page is about? How did you come to that conclusion?,How would you search up videos to learn more about a sustainability habit?,"How can this page improve? What are some interactions or features you would hope to see included here? (And, why?)","Considering this alternate Learn Page, do you prefer this page or the original Learn page? What can the final Learn Page implement from these versions?"
0,HomePage,user1,i can click on my friends and probably message...,it would show me what they achievements they h...,maybe a daily challenge for every day of the week,event images with descriptions of things to do...,some more color i guess this looks intersting ...,7,it makes sense to have a homepage i do not kno...,completed with ease,...,no not really the layers were a bit confusing ...,i guess it did not have the profile edit page ...,10,i think it is packed with a lot of resources a...,i really like this one maybe change the way lo...,i can see this is a videos page because it loo...,this is a page to learn about sustainable habi...,i think you enter your search where it says wa...,a way to filter video content,i like the way the other page let us you learn...
1,HomePage,user2,connect with people w similar interests,add people make new friends,in a week how many plastic bottles can you fil...,probably like garage sales basically swap meet...,colors water bottle background earthy colors e...,7,it is missing colors to attract people and min...,completed with ease,...,no i did not see anything that was not helpful...,just the actual games it would be more fun and...,8,it is good to have an app to involve yourself ...,i think it would be fun and i am looking forwa...,that there is a whole page dedicated to the vi...,it is a learn page,i would use the search bar at the top,it could have options for likeing or reactions...,i like the other page but since i did say it n...


In [13]:
# rename the columns to make it easier to read thru the df
new_column_names = {
    "PageType":"Page Type1",
    "Participant's Name:": "Name", "What options for social interaction do you have using this screen?":"InteractionOptions", 
    "What would you expect to happen next, after tapping an icon in “connect with friends”?":"ConnectIcon_Feature", 
    "What would you like to see embedded in the “Weekly challenges” section? ":"WeeklyChallenges_Feature", 
    "What would you like to see embedded in the “explore local events” section? ":"LocalEvents_Feature", 
    "How can this page improve? What changes or additions would you make to the current Home page?":"Page_Improvements", 
    "How effective is this page in supporting your sustainability journey on this app?\n(More on scale: the middle is somewhat useful in its current state, needs improvement)":"Page_Rating1", 
    "Explain your rating":"Rating_Explanation1",
    "Interact with the Home Page to navigate to your Profile Page.": "Nav_Task1",
    "What information can learn about your account on this page?":"AccountInfo",
    'Does the “Statistics” section display all the information you wish others knew about you at first glance? (why?)':"Statisitcs_Feature",
    'What do you think about the "Friend Suggestions" feature? ':"Friend_Feature",
    "How can this page improve? What changes or additions would you make to the current Profile page?":"Page_Improvements",
    "How effective is this Profile page in supporting your sustainability journey on this app?\n(More on scale: the middle is somewhat useful in its current state, needs improvement)":"Page_Rating2",
    "Explain your rating.":"Rating_Explanation2", 
    "Interact with the Profile Page so you can enter Edit Profile mode.":"Nav_Task2",
    "What options do you have for managing your account on this page?":"ManageOptions",
    "What do you imagine the feature “Export Data” means?":"ExportData_Feature",
    'How would you rate the importance of including a feature, like "Export Data" that allows access to your data?\n(More on Scale: the middle would be it is important but should be implemented a way other than "Export Data")':"Feature_Rating",
    'Why did you choose this rating?':"Rating_Explanation3",
    'What other important security features would you like access to on this page?':"Security_Feature",
    'Exit your Profile and navigate back to your Home Page':"Interaction_Task",
    '(From Home Page) Navigate to the World Map':"Nav_Task3",
    'What are your options for exploring the map on this page?':"Page_Interaction_Options",
    'Examine the icons on the right, what do you think each of them means?':"Filter_Feature",
    'Do the use of these filter icons on the map support your goals within the app? ':"Filter_Feature_Feedback",
    'Use the shirt icon to filter the map (includes a follow up)':"Filter_Task1",
    '(Follow Up) What information is displayed now?':"Task_Feedback",
    'How would you now generate directions to that location? ':"Directions_Feature",
    'Disable the shirt filter to show the default screen on the Map.':"Filter_Task2",
    "Toggle the map’s layers icon and just describe what you see in each layer. What do you think each layer means? (Do not interact with content in each layer yet)":"Layers_Feature",
    'For designer only\nHow did the user interact to toggle World Map layers? What action did they do':"Feature_Interaction_Options",
    'People Layer: \nInteract with the icons embedded in this layer and tell me what options for interaction you have? what information do you learn?':"Layers_Task1",
    'Obstacle Layer:\nInteract with the highlighted regions embedded in this layer and tell me what options for interaction you have? what information do you learn?':"Layers_Task2",
    'Toggle back to the default layer on the map':"Layers_Task3",
    'What other layers were you hoping to find? Would you include anything else in the existing layers? ':"Layers_Feature_Feedback1",
    'Are these layers helpful to your interactions with the map? Helpful in accomplishing your sustainability goals?':"Layers_Feature_Feedback2",
    'How would you rate the importance of including a feature, like "layers" that allows you to access alternate versions of the World Map?\n(More on Scale: the middle would be it is important but needs improvement)':"Layers_Feature_Rating",
    'Explain your rating.1':"Feature_Rating_Explanation4",
    'How effective is the World Map in supporting your sustainability goals?\n(More on Scale: the middle would be it is somewhat effective in its current state but needs improvement)':"Page_Rating3",
    'Explain your rating.2':"Page_Rating_Explanation5",
    'From the World Map, navigate to your Messages ':"Nav_Task4",
    'What do you see on your Messaging page, what do you learn about the types of messages you have? ':"Page_Interaction_Options1",
    'What options are available to you for interacting with others? For instance, what kind of messages can you make and how?':"Page_Interaction_Options2",
    'Do you find this Message page layout helpful for interacting with friends and community? Would you expect to see the Community Board somewhere else?':"Page_Feedback1",
    'What other options for socializing or instant messaging were you hoping to find here? ':"Page_Feedback2",
    'Go back to the World Map page':"Nav_Task5",
    'Navigate to a page where you could find your achievements':"Nav_Task6",
    'What can you infer this page is about? What information is displayed here?':"Page_Interaction_Options",
    'Explain how you would expect to log your achievements and frustrations onto this page?':"Feature_Feedback1",
    'Would a log be useful for establishing short-term sustainability goals? Or would you prefer the app learn about your ongoing short-term goals a different way? (how?)':"Feature_Feedback2",
    'Do you find the "Frustrations" section useful for defining your next goal setting or targeted learning?':"Frustrations_Feature",
    'How would you imagine the entries on this page influence what you see on your Learning page?':"Page_Feedback",
    'How could this page improve to help you feel motivated in achieving goals? Would implementing a personalized daily affirmation be appropriate on this page?':"Page_Improvements",
    'How effective is the Log/Achievements Page in supporting your sustainability goals?\n(More on Scale: the middle would be it is somewhat effective in its current state but needs improvement)':"Page_Rating4",
    'Explain your rating.3':"Rating_Explanation6",
    'Navigate to your Learn page':"Nav_Task7",
    'What information about your learning progress can you find here?':"Page_Interaction_Options",
    'How would you discover the exploratory learning feed? What are your thoughts on this type of layout for a learning module?':"Learn_Feature",
    'What options do you have for curating the learning feed? ':"Learn_Feature_Feedback",
    ' How can you track your progress made on a module?':"Progress_Feature",
    ' Hypothetically, if the "launch quiz" were to become available once you reach 100% complete on a module. Would you feel motivated to interact with the quiz? Would you find that method helpful in supporting or guiding your learning? ':"Quiz_Feature",
    'What are other ways you would like to test your knowledge? What other features could the Learn page implement to motivate learning?':"Page_Improvements",
    'How effective is the Learn Page in supporting your sustainability goals?\n(More on Scale: the middle would be it is somewhat effective in its current state but needs improvement)':"Page_Rating5",
    'Explain your rating.4':"Rating_Explanation7",
    'Consider this alternate Learn Page, what are some features or design choices that stand out to you about this page?':"Page_Feedback",
    'What can you infer this page is about? How did you come to that conclusion?':"Page_Interaction_Options",
    'How would you search up videos to learn more about a sustainability habit?':"Search_Feature",
    'How can this page improve? What are some interactions or features you would hope to see included here? (And, why?)':"Page_Improvements",
    'Considering this alternate Learn Page, do you prefer this page or the original Learn page? What can the final Learn Page implement from these versions?':"Page_Preference"
                                    }
DCO_df.rename(columns=new_column_names, inplace=True)

In [14]:
DCO_df.columns

Index(['Page Type1', 'Name', 'InteractionOptions', 'ConnectIcon_Feature',
       'WeeklyChallenges_Feature', 'LocalEvents_Feature', 'Page_Improvements',
       'Page_Rating1', 'Rating_Explanation1', 'Nav_Task1', 'AccountInfo',
       'Statisitcs_Feature', 'Friend_Feature', 'Page_Improvements',
       'Page_Rating2', 'Rating_Explanation2', 'Nav_Task2', 'ManageOptions',
       'ExportData_Feature', 'Feature_Rating', 'Rating_Explanation3',
       'Security_Feature', 'Interaction_Task', 'Nav_Task3',
       'Page_Interaction_Options', 'Filter_Feature', 'Filter_Feature_Feedback',
       'Filter_Task1', 'Task_Feedback', 'Directions_Feature', 'Filter_Task2',
       'Layers_Feature', 'Feature_Interaction_Options', 'Layers_Task1',
       'Layers_Task2', 'Layers_Task3', 'Layers_Feature_Feedback1',
       'Layers_Feature_Feedback2', 'Layers_Feature_Rating',
       'Feature_Rating_Explanation4', 'Page_Rating3',
       'Page_Rating_Explanation5', 'Nav_Task4', 'Page_Interaction_Options1',
       'Pag

In [15]:
# add Page Type as a column break in the original df
# add Name column after Page Type column to set up the df to be split into smaller dfs
# based on Page Type soon 
page_type = 'ProfilePage'
DCO_df.insert(10, 'PageType2', page_type)
DCO_df.insert(11, 'DuplicateName', DCO_df['Name'])
page_type = 'EditProfilePage'
DCO_df.insert(19, 'PageType3', page_type)
DCO_df.insert(20, 'DuplicateName2', DCO_df['Name'])
page_type = 'WorldMapPage'
DCO_df.insert(28, 'PageType4', page_type)
DCO_df.insert(29, 'DuplicateName3', DCO_df['Name'])
page_type = 'MessagePage'
DCO_df.insert(49, 'PageType5', page_type)
DCO_df.insert(50, 'DuplicateName4', DCO_df['Name'])
page_type = 'AchievementsPage'
DCO_df.insert(57, 'PageType6', page_type)
DCO_df.insert(58, 'DuplicateName5', DCO_df['Name'])
page_type = 'LearnPage'
DCO_df.insert(68, 'PageType7', page_type)
DCO_df.insert(69, 'DuplicateName6', DCO_df['Name'])
page_type = 'Summary Feedback'
DCO_df.insert(78, 'PageType8', page_type)
DCO_df.insert(79, 'DuplicateName7', DCO_df['Name'])
page_type = 'AltLearnPage'
DCO_df.insert(87, 'PageType9', page_type)
DCO_df.insert(88, 'DuplicateName8', DCO_df['Name'])
#check to see if it worked
DCO_df.columns

Index(['Page Type1', 'Name', 'InteractionOptions', 'ConnectIcon_Feature',
       'WeeklyChallenges_Feature', 'LocalEvents_Feature', 'Page_Improvements',
       'Page_Rating1', 'Rating_Explanation1', 'Nav_Task1', 'PageType2',
       'DuplicateName', 'AccountInfo', 'Statisitcs_Feature', 'Friend_Feature',
       'Page_Improvements', 'Page_Rating2', 'Rating_Explanation2', 'Nav_Task2',
       'PageType3', 'DuplicateName2', 'ManageOptions', 'ExportData_Feature',
       'Feature_Rating', 'Rating_Explanation3', 'Security_Feature',
       'Interaction_Task', 'Nav_Task3', 'PageType4', 'DuplicateName3',
       'Page_Interaction_Options', 'Filter_Feature', 'Filter_Feature_Feedback',
       'Filter_Task1', 'Task_Feedback', 'Directions_Feature', 'Filter_Task2',
       'Layers_Feature', 'Feature_Interaction_Options', 'Layers_Task1',
       'Layers_Task2', 'Layers_Task3', 'Layers_Feature_Feedback1',
       'Layers_Feature_Feedback2', 'Layers_Feature_Rating',
       'Feature_Rating_Explanation4', 'Page

In [16]:
# make a subsets of the DCO df 
# so that you can do text sentiment analysis on smaller dfs by slicing the df
# and then go back to cohesive large df to do sentiment analysis again
# this ensures you capture the overarching sentiment a user has during testing
# that will be overlooked when you examine sentiment by page

In [17]:
# split df into separate df subsets by page type
HP_df = DCO_df.loc[:, :'Nav_Task1']
P_df = DCO_df.loc[:, 'PageType2':'Nav_Task2']
EP_df = DCO_df.loc[:, 'PageType3':'Nav_Task3']
WM_df = DCO_df.loc[:, 'PageType4':'Nav_Task4']
M_df = DCO_df.loc[:, 'PageType5':'Nav_Task6']
A_df = DCO_df.loc[:, 'PageType6':'Nav_Task7']
L_df = DCO_df.loc[:, 'PageType7':'Rating_Explanation7']
AL_df = DCO_df.loc[:, 'PageType9':'Page_Preference']

Check the subset dfs are correct by having  quick look at the head

In [18]:
#HP_df.head(1)
#P_df.head(1)
#EP_df.head(1)
#M_df.head(1)
#WM_df.head(1)
#A_df.head(1)
#L_df.head(1)
AL_df.head(1)

,PageType9,DuplicateName8,Page_Feedback,Page_Interaction_Options,Search_Feature,Page_Improvements,Page_Preference
0,AltLearnPage,user1,i can see this is a videos page because it loo...,this is a page to learn about sustainable habi...,i think you enter your search where it says wa...,a way to filter video content,i like the way the other page let us you learn...


In [19]:
# Navigation Task to World Map should be at the end of the Home Page
# move the Nav_Task3 column from Edit Profile Page to end of Home Page
column_to_move = "Nav_Task3"

# Pop the column from the end of EP_df
moved_column = EP_df.pop(column_to_move)

# Insert the popped column at the end of HP_df
HP_df.insert(len(HP_df.columns), column_to_move, moved_column)

#check that column is moved 
HP_df.head(1)

,Page Type1,Name,InteractionOptions,ConnectIcon_Feature,WeeklyChallenges_Feature,LocalEvents_Feature,Page_Improvements,Page_Rating1,Rating_Explanation1,Nav_Task1,Nav_Task3
0,HomePage,user1,i can click on my friends and probably message...,it would show me what they achievements they h...,maybe a daily challenge for every day of the week,event images with descriptions of things to do...,some more color i guess this looks intersting ...,7,it makes sense to have a homepage i do not kno...,completed with ease,completed with ease


In [20]:
# follow up from above
EP_df.head(1)

,PageType3,DuplicateName2,ManageOptions,ExportData_Feature,Feature_Rating,Rating_Explanation3,Security_Feature,Interaction_Task
0,EditProfilePage,user1,you could add a profile pic name username and ...,you could remove all data before you delete yo...,10,i like that it is not hidden it makes the app ...,you should not be able to change name and emai...,completed with ease


In [21]:
# Navigation Task to Achievements should be at the end of the World Map Page
# move the Nav_Task6 column from Message Page to end of World Map Page
column_to_move = "Nav_Task6"

# Pop the column from the end of M_df
moved_column = M_df.pop(column_to_move)

# Insert the popped column at the end of HP_df
WM_df.insert(len(WM_df.columns), column_to_move, moved_column)

#check that column is moved 
WM_df.head(1)

,PageType4,DuplicateName3,Page_Interaction_Options,Filter_Feature,Filter_Feature_Feedback,Filter_Task1,Task_Feedback,Directions_Feature,Filter_Task2,Layers_Feature,...,Layers_Task2,Layers_Task3,Layers_Feature_Feedback1,Layers_Feature_Feedback2,Layers_Feature_Rating,Feature_Rating_Explanation4,Page_Rating3,Page_Rating_Explanation5,Nav_Task4,Nav_Task6
0,WorldMapPage,user1,explore events in the search bar,t shirt maybe means free clothes teardrop mean...,yeah if it is what i think it is i can see the...,completed with ease,it is telling me where there is more shirts it...,click the shirt and you click the arrow to sta...,completed with some difficulty,i can not find it i am clicking the actual map...,...,i do not understand the color choices i think ...,completed with some difficulty,i can not think of anything,yes because they make you more interactive and...,8,i think its pretty good to consider other thin...,9,with some more color and edits to the way the ...,completed with some difficulty,completed with some difficulty


In [22]:
# follow up from above
M_df.head(1)

,PageType5,DuplicateName4,Page_Interaction_Options1,Page_Interaction_Options2,Page_Feedback1,Page_Feedback2,Nav_Task5
0,MessagePage,user1,there is a beach clean up and i can get more d...,you could set up an event and message people o...,yeah it is really cool and as helpful as the m...,i am not sure but where is the friends page,completed with ease


Now that all the subsets are set and the main columns are renamed, let's rename the subset columns to make them more clear to read

In [23]:
rename_columns = {
    'Page Type1':'Page Type', 
    'Name':'Name', 
    'InteractionOptions':'Interaction_Options', 
    'ConnectIcon_Feature': 'Connect_Feature',
    'WeeklyChallenges_Feature': 'WeeklyChallenges_Feature', 
    'LocalEvents_Feature': 'LocalEvents_Feature', 
    'Page_Improvements':'Improvement_Feedback',
    'Page_Rating1': 'Page_Rating', 
    'Rating_Explanation1':'Rating_Explanation', 
    'Nav_Task1': 'Nav_toMessage_Task', 
    'Nav_Task3': 'Nav_toWorldMap_Task'
}
HP_df.rename(columns=rename_columns, inplace=True)
HP_df.columns

Index(['Page Type', 'Name', 'Interaction_Options', 'Connect_Feature',
       'WeeklyChallenges_Feature', 'LocalEvents_Feature',
       'Improvement_Feedback', 'Page_Rating', 'Rating_Explanation',
       'Nav_toMessage_Task', 'Nav_toWorldMap_Task'],
      dtype='object')

In [24]:
rename_columns = {
    'PageType2': 'Page Type', 
    'DuplicateName': 'Name', 
    'AccountInfo': 'Account_Feedback', 
    'Statisitcs_Feature': 'Stats_Feature',
    'Friend_Feature': 'Friend_Feature', 
    'Page_Improvements': 'Improvement_Feedback', 
    'Page_Rating2': 'Page_Rating',
    'Rating_Explanation2':'Rating_Explanation', 
    'Nav_Task2': 'Nav_toEditProfile_Task'
}
P_df.rename(columns=rename_columns, inplace=True)
P_df.columns

Index(['Page Type', 'Name', 'Account_Feedback', 'Stats_Feature',
       'Friend_Feature', 'Improvement_Feedback', 'Page_Rating',
       'Rating_Explanation', 'Nav_toEditProfile_Task'],
      dtype='object')

In [25]:
rename_columns = {
    'PageType3':'Page Type', 
    'DuplicateName2': 'Name', 
    'ManageOptions': 'Interaction_Options', 
    'ExportData_Feature': 'ExportData_Feature',
    'Feature_Rating':'Feature_Rating', 
    'Rating_Explanation3':'Rating_Explanation', 
    'Security_Feature': 'Security_Feature',
    'Interaction_Task': 'Nav_toHome_Task'
}
EP_df.rename(columns=rename_columns, inplace=True)
EP_df.columns

Index(['Page Type', 'Name', 'Interaction_Options', 'ExportData_Feature',
       'Feature_Rating', 'Rating_Explanation', 'Security_Feature',
       'Nav_toHome_Task'],
      dtype='object')

In [26]:
rename_columns = {
    'PageType5':'Page Type', 
    'DuplicateName4':'Name', 
    'Page_Interaction_Options1': 'Interaction_Options',
    'Page_Interaction_Options2': 'Message_Feature', 
    'Page_Feedback1': 'Message_Feedback', 
    'Page_Feedback2': 'Improvement_Feedback',
    'Nav_Task5': 'Nav_toWorldMap_Task'
}
M_df.rename(columns=rename_columns, inplace=True)
M_df.columns

Index(['Page Type', 'Name', 'Interaction_Options', 'Message_Feature',
       'Message_Feedback', 'Improvement_Feedback', 'Nav_toWorldMap_Task'],
      dtype='object')

In [27]:
rename_columns = {
    'PageType4': 'Page Type', 
    'DuplicateName3': 'Name', 
    'Page_Interaction_Options': 'Interaction_Options',
    'Filter_Feature': 'Filter_Feature', 
    'Filter_Feature_Feedback': 'Filter_Feedback', 
    'Filter_Task1': 'StartFilter_Task',
    'Task_Feedback': 'Task_Feedback', 
    'Directions_Feature': 'Directions_Feature', 
    'Filter_Task2': 'EndFilter_Task', 
    'Layers_Feature': 'Layers_Feature',
    'Feature_Interaction_Options': 'Interaction_Type', 
    'Layers_Task1':'PeopleLayers_Feature', 
    'Layers_Task2':'ObstacleLayers_Feature',
    'Layers_Task3': 'Interaction_Task', 
    'Layers_Feature_Feedback1': 'LayersA_Feedback', 
    'Layers_Feature_Feedback2': 'LayersB_Feedback',
    'Layers_Feature_Rating':'Layers_Feature_Rating', 
    'Feature_Rating_Explanation4': 'Layers_Rating_Explanation', 
    'Page_Rating3': 'Page_Rating',
    'Page_Rating_Explanation5': 'Rating_Explanation', 
    'Nav_Task4': 'Nav_toMessage_Task', 
    'Nav_Task6': 'Nav_toAchievementPage_Task'
}
WM_df.rename(columns=rename_columns, inplace=True)
WM_df.columns

Index(['Page Type', 'Name', 'Interaction_Options', 'Filter_Feature',
       'Filter_Feedback', 'StartFilter_Task', 'Task_Feedback',
       'Directions_Feature', 'EndFilter_Task', 'Layers_Feature',
       'Interaction_Type', 'PeopleLayers_Feature', 'ObstacleLayers_Feature',
       'Interaction_Task', 'LayersA_Feedback', 'LayersB_Feedback',
       'Layers_Feature_Rating', 'Layers_Rating_Explanation', 'Page_Rating',
       'Rating_Explanation', 'Nav_toMessage_Task',
       'Nav_toAchievementPage_Task'],
      dtype='object')

In [28]:
rename_columns = {
    'PageType6': 'Page Type', 
    'DuplicateName5': 'Name', 
    'Page_Interaction_Options': 'Interaction_Options',
    'Feature_Feedback1': 'Log_Feature', 
    'Feature_Feedback2':'Log_Feature_Feedback', 
    'Frustrations_Feature':'Frustration_Feature',
    'Page_Feedback':'Page_Feedback', 
    'Page_Improvements': 'Improvement_Feedback', 
    'Page_Rating4': 'Page_Rating',
    'Rating_Explanation6':'Rating_Explanation', 
    'Nav_Task7':'Nav_toLearnPage_Task'
}
A_df.rename(columns=rename_columns, inplace=True)
A_df.columns

Index(['Page Type', 'Name', 'Interaction_Options', 'Log_Feature',
       'Log_Feature_Feedback', 'Frustration_Feature', 'Page_Feedback',
       'Improvement_Feedback', 'Page_Rating', 'Rating_Explanation',
       'Nav_toLearnPage_Task'],
      dtype='object')

In [29]:
rename_columns = {
    'PageType7':'Page Type', 
    'DuplicateName6':'Name', 
    'Page_Interaction_Options':'Interaction_Options',
    'Learn_Feature': 'Learn_Feature', 
    'Learn_Feature_Feedback': 'Learn_Feature_Feedback', 
    'Progress_Feature':'Progress_Feature',
    'Quiz_Feature':'Quiz_Feature', 
    'Page_Improvements':'Improvement_Feedback', 
    'Page_Rating5':'Page_Rating',
    'Rating_Explanation7':'Rating_Explanation'
}
L_df.rename(columns=rename_columns, inplace=True)
L_df.columns

Index(['Page Type', 'Name', 'Interaction_Options', 'Learn_Feature',
       'Learn_Feature_Feedback', 'Progress_Feature', 'Quiz_Feature',
       'Improvement_Feedback', 'Page_Rating', 'Rating_Explanation'],
      dtype='object')

In [30]:
rename_columns = {
    'PageType9':'Page Type', 
    'DuplicateName8':'Name', 
    'Page_Feedback':'Page_Feedback',
    'Page_Interaction_Options':'Interaction_Options', 
    'Search_Feature':'Search_Feature', 
    'Page_Improvements': 'Improvement_Feedback',
    'Page_Preference':'Page_Preference'
}
AL_df.rename(columns=rename_columns, inplace=True)
AL_df.columns

Index(['Page Type', 'Name', 'Page_Feedback', 'Interaction_Options',
       'Search_Feature', 'Improvement_Feedback', 'Page_Preference'],
      dtype='object')

Now let's do some light EDA on the dataframes with ratings.

In [31]:
describe_stats = DCO_df.describe()
describe_stats

,Page_Rating1,Page_Rating2,Feature_Rating,Layers_Feature_Rating,Page_Rating3,Page_Rating4,Page_Rating5,"On a scale of 1-10, how likely are you to use an app developed from this wireframe to meet your sustainability needs and goals?\n(More on Scale: the middle would be I would use a sustainability app, but if it were different than what I saw in this wireframe.)"
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,8.142857,8.142857,8.142857,7.285714,8.714286,8.142857,8.000000,8.142857
std,1.463850,1.463850,1.864454,1.603567,1.603567,1.069045,1.290994,1.345185
min,7.000000,6.000000,6.000000,5.000000,6.000000,7.000000,6.000000,7.000000
25%,7.000000,7.500000,6.500000,6.500000,8.000000,7.500000,7.000000,7.000000
50%,7.000000,8.000000,8.000000,7.000000,9.000000,8.000000,9.000000,8.000000
75%,9.500000,9.000000,10.000000,8.000000,10.000000,8.500000,9.000000,9.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000


In the chart above, from left to right: HP_Rating,P_Rating,EP_Rating,WM_Rating,A_Rating,L_Rating, Overall_Rating 
This is not an efficient way to look at ratings, lets rename the columns and regenerate chart.

In [32]:
rename_columns = {
    'Page_Rating1':'Home Page Rating', 
    'Page_Rating2':'Profile Page Rating',
    'Feature_Rating':'Export Data Feature Rating',
    'Layers_Feature_Rating':'Layers Feature Rating', 
    'Page_Rating3': 'World Map Page Rating', 
    'Page_Rating4': 'Achievement Page Rating', 
    'Page_Rating5': 'Learn Page Rating',
    'On a scale of 1-10, how likely are you to use an app developed from this wireframe to meet your sustainability needs and goals?\n(More on Scale: the middle would be I would use a sustainability app, but if it were different than what I saw in this wireframe.)':'Wireframe Rating'
}
describe_stats.rename(columns=rename_columns, inplace=True)
describe_stats.columns

Index(['Home Page Rating', 'Profile Page Rating', 'Export Data Feature Rating',
       'Layers Feature Rating', 'World Map Page Rating',
       'Achievement Page Rating', 'Learn Page Rating', 'Wireframe Rating'],
      dtype='object')

In [33]:
describe_stats

,Home Page Rating,Profile Page Rating,Export Data Feature Rating,Layers Feature Rating,World Map Page Rating,Achievement Page Rating,Learn Page Rating,Wireframe Rating
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,8.142857,8.142857,8.142857,7.285714,8.714286,8.142857,8.000000,8.142857
std,1.463850,1.463850,1.864454,1.603567,1.603567,1.069045,1.290994,1.345185
min,7.000000,6.000000,6.000000,5.000000,6.000000,7.000000,6.000000,7.000000
25%,7.000000,7.500000,6.500000,6.500000,8.000000,7.500000,7.000000,7.000000
50%,7.000000,8.000000,8.000000,7.000000,9.000000,8.000000,9.000000,8.000000
75%,9.500000,9.000000,10.000000,8.000000,10.000000,8.500000,9.000000,9.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000


From the chart above we can see that on average the highest rating page was the World Map Page, and overall the other pages were around the same rating. The holistic rating for this wireframe is also around the same rating of 8.14/10, 10 being the highest favorable score.

To make our text data exploration easier, we will generalize the response per user for each Home Page question by sentiment categorizations such as: positive, neutral, negative.
And choose what to further examine based on those sentiments.

In [37]:
from nltk.sentiment import SentimentIntensityAnalyzer

def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(str(text))['compound']
    
    # Classify sentiment as 'positive', 'negative', or 'neutral' based on the compound score
    if sentiment_score > 0:
        return 'positive'
    elif sentiment_score < 0:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis function to all cells in the DataFrame
sentiment_HP_df = HP_df.applymap(analyze_sentiment)
sentiment_HP_df
# below sentiment analysis matrix for all Home Page responses!

,Page Type,Name,Interaction_Options,Connect_Feature,WeeklyChallenges_Feature,LocalEvents_Feature,Improvement_Feedback,Page_Rating,Rating_Explanation,Nav_toMessage_Task,Nav_toWorldMap_Task
0,neutral,neutral,positive,positive,positive,positive,positive,neutral,positive,positive,positive
1,neutral,neutral,positive,positive,neutral,positive,neutral,neutral,positive,positive,positive
2,neutral,neutral,positive,neutral,positive,neutral,positive,neutral,positive,positive,positive
3,neutral,neutral,positive,neutral,positive,neutral,positive,neutral,positive,positive,positive
4,neutral,neutral,positive,positive,positive,positive,positive,neutral,positive,positive,positive
5,neutral,neutral,positive,positive,positive,positive,positive,neutral,neutral,positive,positive
6,neutral,neutral,positive,positive,positive,neutral,positive,neutral,negative,positive,positive


In [38]:
sentiment_P_df = P_df.applymap(analyze_sentiment)
sentiment_P_df
# below sentiment analysis matrix for all Profile Page responses!

,Page Type,Name,Account_Feedback,Stats_Feature,Friend_Feature,Improvement_Feedback,Page_Rating,Rating_Explanation,Nav_toEditProfile_Task
0,neutral,neutral,positive,negative,positive,positive,neutral,positive,negative
1,neutral,neutral,positive,positive,positive,positive,neutral,positive,negative
2,neutral,neutral,positive,positive,positive,positive,neutral,positive,positive
3,neutral,neutral,positive,neutral,positive,positive,neutral,positive,positive
4,neutral,neutral,positive,positive,positive,positive,neutral,positive,positive
5,neutral,neutral,positive,positive,positive,positive,neutral,positive,positive
6,neutral,neutral,positive,positive,positive,negative,neutral,positive,positive


In [39]:
sentiment_EP_df = EP_df.applymap(analyze_sentiment)
sentiment_EP_df
# below sentiment analysis matrix for all Edit Profile Page responses!

,Page Type,Name,Interaction_Options,ExportData_Feature,Feature_Rating,Rating_Explanation,Security_Feature,Nav_toHome_Task
0,neutral,neutral,neutral,neutral,neutral,positive,positive,positive
1,neutral,neutral,neutral,positive,neutral,neutral,positive,positive
2,neutral,neutral,negative,negative,neutral,positive,positive,positive
3,neutral,neutral,neutral,positive,neutral,positive,neutral,positive
4,neutral,neutral,neutral,positive,neutral,positive,positive,positive
5,neutral,neutral,neutral,neutral,neutral,positive,positive,negative
6,neutral,neutral,neutral,negative,neutral,positive,neutral,neutral


In [40]:
sentiment_WM_df = WM_df.applymap(analyze_sentiment)
sentiment_WM_df
# below sentiment analysis matrix for all World Map Page responses!

,Page Type,Name,Interaction_Options,Filter_Feature,Filter_Feedback,StartFilter_Task,Task_Feedback,Directions_Feature,EndFilter_Task,Layers_Feature,...,ObstacleLayers_Feature,Interaction_Task,LayersA_Feedback,LayersB_Feedback,Layers_Feature_Rating,Layers_Rating_Explanation,Page_Rating,Rating_Explanation,Nav_toMessage_Task,Nav_toAchievementPage_Task
0,neutral,neutral,neutral,positive,positive,positive,positive,negative,negative,neutral,...,positive,negative,neutral,positive,neutral,positive,neutral,positive,negative,negative
1,neutral,neutral,positive,negative,positive,positive,neutral,positive,negative,neutral,...,positive,negative,neutral,positive,neutral,positive,neutral,positive,positive,positive
2,neutral,neutral,positive,neutral,positive,positive,neutral,neutral,positive,negative,...,neutral,positive,negative,positive,neutral,negative,neutral,negative,positive,positive
3,neutral,neutral,positive,neutral,positive,positive,neutral,positive,positive,neutral,...,positive,positive,negative,positive,neutral,neutral,neutral,positive,positive,positive
4,neutral,neutral,positive,neutral,positive,positive,positive,positive,positive,positive,...,positive,positive,positive,positive,neutral,positive,neutral,positive,positive,positive
5,neutral,neutral,neutral,neutral,positive,positive,neutral,neutral,positive,positive,...,positive,positive,positive,positive,neutral,positive,neutral,positive,positive,positive
6,neutral,neutral,neutral,positive,positive,positive,neutral,negative,neutral,negative,...,positive,negative,negative,positive,neutral,negative,neutral,positive,negative,positive


In [41]:
sentiment_M_df = M_df.applymap(analyze_sentiment)
sentiment_M_df
# below sentiment analysis matrix for all Message Page responses!

,Page Type,Name,Interaction_Options,Message_Feature,Message_Feedback,Improvement_Feedback,Nav_toWorldMap_Task
0,neutral,neutral,positive,neutral,positive,positive,positive
1,neutral,neutral,positive,positive,positive,positive,positive
2,neutral,neutral,neutral,positive,positive,neutral,positive
3,neutral,neutral,neutral,positive,neutral,neutral,positive
4,neutral,neutral,positive,neutral,positive,positive,positive
5,neutral,neutral,neutral,neutral,positive,positive,positive
6,neutral,neutral,neutral,positive,positive,negative,positive


In [42]:
sentiment_A_df = A_df.applymap(analyze_sentiment)
sentiment_A_df
# below sentiment analysis matrix for all Achievement Page responses!

,Page Type,Name,Interaction_Options,Log_Feature,Log_Feature_Feedback,Frustration_Feature,Page_Feedback,Improvement_Feedback,Page_Rating,Rating_Explanation,Nav_toLearnPage_Task
0,neutral,neutral,negative,positive,positive,positive,neutral,positive,neutral,positive,positive
1,neutral,neutral,negative,positive,positive,positive,neutral,positive,neutral,positive,positive
2,neutral,neutral,negative,neutral,positive,positive,neutral,positive,neutral,positive,positive
3,neutral,neutral,positive,neutral,positive,negative,negative,neutral,neutral,positive,positive
4,neutral,neutral,positive,positive,positive,positive,positive,positive,neutral,positive,positive
5,neutral,neutral,negative,neutral,positive,positive,negative,positive,neutral,positive,positive
6,neutral,neutral,neutral,positive,positive,positive,positive,positive,neutral,neutral,positive


In [43]:
sentiment_L_df = L_df.applymap(analyze_sentiment)
sentiment_L_df
# below sentiment analysis matrix for all Learn Page responses!

,Page Type,Name,Interaction_Options,Learn_Feature,Learn_Feature_Feedback,Progress_Feature,Quiz_Feature,Improvement_Feedback,Page_Rating,Rating_Explanation
0,neutral,neutral,positive,positive,positive,positive,positive,positive,neutral,positive
1,neutral,neutral,neutral,positive,positive,positive,positive,positive,neutral,positive
2,neutral,neutral,neutral,positive,negative,positive,positive,positive,neutral,positive
3,neutral,neutral,neutral,neutral,positive,positive,positive,positive,neutral,neutral
4,neutral,neutral,positive,positive,positive,positive,negative,positive,neutral,positive
5,neutral,neutral,positive,positive,neutral,positive,negative,positive,neutral,positive
6,neutral,neutral,neutral,negative,neutral,positive,negative,positive,neutral,positive


Now let's examine each subset df in depth

In [44]:
pd.set_option('display.max_colwidth', None)
HP_df

,Page Type,Name,Interaction_Options,Connect_Feature,WeeklyChallenges_Feature,LocalEvents_Feature,Improvement_Feedback,Page_Rating,Rating_Explanation,Nav_toMessage_Task,Nav_toWorldMap_Task
0,HomePage,user1,i can click on my friends and probably message them i can see my weekly challenges and explore local events although i think that should go on the map page instead,it would show me what they achievements they have made and allow me to message them,maybe a daily challenge for every day of the week,event images with descriptions of things to do in my area that i can join or sign up for,some more color i guess this looks intersting but pretty dull,7,it makes sense to have a homepage i do not know what else i would add here aside from what i said before but i like the way the page looks overall it looks clean and like what a homepage should look like on a social media,completed with ease,completed with ease
1,HomePage,user2,connect with people w similar interests,add people make new friends,in a week how many plastic bottles can you fill collect,probably like garage sales basically swap meet for thrifting basically places to sell or trade with people like poshmark,colors water bottle background earthy colors ecosystem themed,7,it is missing colors to attract people and mini games to reach goals but that is more later the home page shows what you need to do and connect with other people,completed with ease,completed with ease
2,HomePage,user3,connecting with friends doing weekly challenges and exploring local events,either bring me to their profiles page or a direct messages page,i would like to see a progress bar shows how many weekly challenges i have completed towards my goal,information about the date time and location of local sustainability events that i can attend,i feel like the navigation icons are kind of vague i would like to see more commonly used icons or labels that can make it easier to navigate,7,i think it could benefit from a section that relates to my personal sustainability goals i would like to see a more personalized home page,completed with ease,completed with ease
3,HomePage,user4,connecting with friends weekly challenges and explore local events,a page to see people s profiles and options if we can connect with them or not,pictures depicting new challenges every week and descriptions for each event,pictures showing what events are happening and descriptions for each event,i think this page can improve by giving a recap on your sustainability journey like a ring circle to see what levels you are on,7,i like how it gives you options to help you be more sustainable but i would like a visual on what my current sustainability journey is,completed with ease,completed with ease
4,HomePage,user5,i can click on different pages there different icons like the map learn tab and achievements,i would expect the profile to pop up and option to add as friend or message,things to do in the area suggestions like guides,i would like to see farmer s markets sustainable locations events and volunteering opportunities or sustainable options in your area activities like thrifting or beach clean ups,the events should have clear dates and the weekly challenges should be able to show the rewards system,10,all the opportunities available and i always default to the home page when i am lost on a social app so i think this page is needed you always need a home page to summarize interactions and suggest other interactions,completed with ease,completed with ease
5,HomePage,user6,connect with friends and explore local events,i would expect to see contacts of my friends,i would like to see images related to the challenges and rankings,i would like to see instructions on how to participate in the events,i think the home page is well done already one thing that could be improved is to have more features such as current news related to the topics and a how to guide in using the app,10,the home page is crucial for allowing the user to explore

In [45]:
P_df

,Page Type,Name,Account_Feedback,Stats_Feature,Friend_Feature,Improvement_Feedback,Page_Rating,Rating_Explanation,Nav_toEditProfile_Task
0,ProfilePage,user1,i am not sure what that means because you can not learn much you can only see my achievements and sustainable locations but i do not know what sustainable locations means so i do not know what i would learn from that i do not think the achievements makes sense either since they are my personal achievements unless it means like awards that i have i guess you could learn how many friends i have and a streak but a streak is more personal incentive so you can not learn much there is no bio i do not know,no i do not think the statistics section makes sense wiht what is shown for achievements and streak i think streak page is different from statisitics so no it does not show what i would want others to know about me i would not feel comfortable sharing my current location so if that is what sustainable locations means i would not want that i think this should say what i am about in sustainability and maybe what i am looking for,i think the friends suggestions does not make sense here if other people can see that on my personal profile page i would not want them to see that just how many i have or not at all if its just visible on my end i think coming all the way to profile page to see suggestions is too far i guess friend suggestions makes sense but the other sections on the profile do not make sense it should be more simple like you should not have to interact with this page too much unless you are changing your personal information it should be more static and the focus should be on the home page and world map where you actually have social interactions,remove the statistics section and add a personal bio and let people choose if they share their location or not for safety reasons you do not want to expose someone,6,i think it is a page that is necessary because how else will you edit name and pronoun or relevant demographic information but it needs to still be improved by getting rid of certain sections,completed with some difficulty
1,ProfilePage,user2,it shows when i joined how many friends some achievements and my streaks so if i want to do better i can see my streaks and sustainable locations where i interacted,maybe i would like to have a bio so i can say what i like to do so if i like to thrift or go to beach clean ups,i think that one is cool because you get to see who else you have in common you can closer to friends and family and you get support because you are not the only one using this app,color add bio i like the locations it makes sense though,8,same as above i liked the locations but needs color and a bio,completed with some difficulty
2,ProfilePage,user3,when i joined the app how many friends i have my sustainability stats my achievements,no i wish it could tell others what i care about those most e g this user cares most about environmental sustainability,i think it is a good feature because it could be hard to find friends without suggestions especially in the beginning,i wish it would be more specific about what sustainable locations means,8,i think this page must be included on the final app but i would not say it is extremely effective because there are more features that can be implemented into it,completed with ease
3,ProfilePage,user4,when you joined how many friends you have how long you have been sustainable locations they can help you be sustainable and your achievements,what does sustainable locations mean and what is it supposed to tell people,i like the friends suggestion feature but i think the placement is a little odd out of nowhere i think it would be better if it was on a page that displays all your friends and then you are given an options to make more friends,i would suggest to move the friend suggestion to somewhere else because this page seems more for recapping your profile not to find new friends,8,gives you a good recap on your jour

In [46]:
EP_df

,Page Type,Name,Interaction_Options,ExportData_Feature,Feature_Rating,Rating_Explanation,Security_Feature,Nav_toHome_Task
0,EditProfilePage,user1,you could add a profile pic name username and you could delete your account i think this makes more sense for a profile,you could remove all data before you delete your account or transfer your data,10,i like that it is not hidden it makes the app more transparent in privacy practices also if you can export all your data you can save everything you do not have to keep your account this makes it easy and i do not feel tied down to the app,you should not be able to change name and email for safety reasons like fake accounts,completed with ease
1,EditProfilePage,user2,i get to put my name username choose a profile pic,i think its to probably transfer my habits to someone else so for sharing data,7,people should see what new app you have found and it would lure more people in,probably like you could switch it to private or public and you could choose your own avatar also a bio,completed with ease
2,EditProfilePage,user3,change profile picture change name change username change email logout export data not sure what this means delete your account,i am not sure but probably would have something to do with exporting the sustainability streaks that i have had or the locations i have visited,6,i feel like i would not have too much use for exporting data but it would be a cool feature to have,definitely a password option maybe like a face id for logging in option,completed with ease
3,EditProfilePage,user4,name profile picture username and email,save your sustainability journey to a file,10,good to have an option to save your data somewhere else,change password option,completed with ease
4,EditProfilePage,user5,youy can change name username delete account,i am not sure maybe to share with other people or for sharing profile,6,i think its cool if it is to keep access of your data it does not hurt i would just go to my achievements though and check my data history there i would not worry about my data access it can stay but it is not a user need,is there any subscriptions if this app had subscriptions i would want to be able to manage that here,completed with ease
5,EditProfilePage,user6,there are options to change the profile picture name username and email you can also logout of the account export data or delete account,i imagine this feature to be an export of the user s profile data along with the statistics from the previous page,8,i think this feature is important for users to keep the data of their interaction with the app but it is not as crucial compared to the previous ones,i would like to have access to adding a two factor security code or something like a backup email or phone number to access the account,completed with some difficulty
6,EditProfilePage,user7,name username email,transfer data from app to another phone if you lose your current phone,10,people would want to keep their progress,does not need anything else,did not complete task


In [47]:
WM_df

,Page Type,Name,Interaction_Options,Filter_Feature,Filter_Feedback,StartFilter_Task,Task_Feedback,Directions_Feature,EndFilter_Task,Layers_Feature,...,ObstacleLayers_Feature,Interaction_Task,LayersA_Feedback,LayersB_Feedback,Layers_Feature_Rating,Layers_Rating_Explanation,Page_Rating,Rating_Explanation,Nav_toMessage_Task,Nav_toAchievementPage_Task
0,WorldMapPage,user1,explore events in the search bar,t shirt maybe means free clothes teardrop means water or water house the next is recycling center the leaf bin i am not sure maybe those green trash cans so where you throw away grass and yard clippings,yeah if it is what i think it is i can see there are friends and stacks i can find all the resources and people on here,completed with ease,it is telling me where there is more shirts it will show me where to get free clothes or sustainable clothes,click the shirt and you click the arrow to start directions however i do not like that this just shows a goodwill i thought it would show more like clothes exchange or free clothes locations i think there should be a separate event filter i expect to see places where i could donate clothes but more grassroots resources or locations like veterans boxes swap meets school drop boxes etc,completed with some difficulty,i can not find it i am clicking the actual map interface oh i found it but i do not understand it yet i see a purple and yellow spot,...,i do not understand the color choices i think it is a heat map for popular places i see the huge area is just an explore area and i would get treasures i see that you can practice sustainability it is self explanatory and i can accept a challenge,completed with some difficulty,i can not think of anything,yes because they make you more interactive and keep you interested,8,i think its pretty good to consider other things users could be interested in not just resources so its like a bonus on the map and it is not too cluttered because you can unlock that layer and it is not the default but i think the second layer with the colors needs improvements because i do not know what it means or how important it is that i interact or visit those locations my favorite was the challenge spot that one made more sense,9,with some more color and edits to the way the interactions flow i think it is very good i would still use an app just for the map right now as it is it has a lot of useful resource information that is hard to find just by google searches and this takes out all the work from that so the map is the most important feature on this app however i was dissappointed to see that for the locations it was just showing goodwill because i could have just easily found that on my phone map i think the whole point of this technology would be to provide resources for underground events and organizations like swap meets free clothes trading posts and food banks so i think it needs to include that over known locations because that is what makes it more communal and interesting over what is out there right now i also would like to see options for transportation like car pooling requests or the bus,completed with some difficulty,completed with some difficulty
1,WorldMapPage,user2,i see how there is a lot of people events recycle shirt where my friends are located so i can be there too,clothes water and recycle and green waste,yes you can use those or make your own to find habits people,completed with ease,it shows it filters where people are selling clothes or yard sales and houses,yes it has a little location thing to see how far or how long it would take me,completed with a lot of difficulty,i see people on the map,...,it shows where you can unlock a new city basically it helps build my recycling skills,completed with some difficulty,a layer that shows how many points i have so far from going to different places,yes they do because i want to know where people are and what benefits i am getting and if its a real place or not,7,very important because it shows unl

In [48]:
M_df

,Page Type,Name,Interaction_Options,Message_Feature,Message_Feedback,Improvement_Feedback,Nav_toWorldMap_Task
0,MessagePage,user1,there is a beach clean up and i can get more details i can see opened messages and community events sign up it is cool because this feels like a fun and better informed way of finding events but still ensures safety measures because you actually have to sign up it feels legitimate and only actual users post here i think to make it more safe the community board events should be generated only by users that are verified or already existing organizations or someone with a title or experience maybe for some posts on the board they can be visible to public friends or close friends,you could set up an event and message people one on one,yeah it is really cool and as helpful as the map and i like that it is a community board on a digital space and it is helpful things only no irrelevant or personal posts there is less room for trolling and more positive support,i am not sure but where is the friends page,completed with ease
1,MessagePage,user2,it shows me the community ones what individual users have said to me and where i can sign up or join events,i can make any messages to my friends probably ones if i am at the event or not where i will present my clothes at,i think its better to have personal and community messages on the same page because it is doing the same thing,probably have somewhere where i can like requests to add a little emoji or reaction to their messages,completed with ease
2,MessagePage,user3,i see users who have messaged me i can see what category the messages relate to,i can add new messages and i can signup or join events,i think it is good to have messages for interactions but the community board is kind of unexpectedly there,i was just expecting a normal instant messages page,completed with ease
3,MessagePage,user4,i learned that i have an unread message and locations relating to the message,i can join messages with other users regarding a task and i can make new messages by pressing the plus button,i expect it to be somewhere else because i assumed the message page would be already existing messages i would have with people,none,completed with ease
4,MessagePage,user5,i can get messages from the app itself for details of an event and friends can message you i think the community board is cool i like it,group chat with community board and personal messages,i think i did not expect the messages icon on the world map personally i would like it in a separate tab because it has a lot of stuff and its important stuff like the community board is necessary and it makes sense to be in the messages but it is not necessary to have the message symbol on the map because the notifications would tell you to read a message and can just redirect you to the messages it does not need to take up space on the map because there is a lot of interactions you can do on that page already on its own the messages would also be easier to navigate to if it had its own icon symbol at the bottom with the other pages,i would like an option to see all my friends because right now i can see they message me but i can not scroll through friends list so you need options to text friends by scrolling through a friend or contact list,completed with ease
5,MessagePage,user6,i learn about details of events direct messages from users and how many users have sent me a message there is also a community board,i can direct message others and joining a community board to talk of multiple users at once,i think the layout is simple and direct i would expect to see the community board in the home page as well,i was hoping to see what happens when i click on the plus sign,completed with ease
6,MessagePage,user7,bonfires and beach cleanups and events and users messaging you meeting up with other people for sustainability practices,opportunities to practice sustainability message others nearby food cleaning,yes it should have been at th

In [49]:
A_df

,Page Type,Name,Interaction_Options,Log_Feature,Log_Feature_Feedback,Frustration_Feature,Page_Feedback,Improvement_Feedback,Page_Rating,Rating_Explanation,Nav_toLearnPage_Task
0,AchievementsPage,user1,to log your achievements and frustrations it is for daily habits,i would like to click popular goals and frustrations based on experience level like pre determined categories because typing in is too much like a journal and is not doing anything new i would prefer it to auto generate my goals and then auto curate my learn feed,a log is useful because there needs to be some kind of way to update your immediate goals and they should change as you become more experienced,sort of i think but it is easier to think in terms of goals because would not your frustrations just become your goals,the entries should automatically influence the kind of content i see on the learn page also based on what other people in my experience level are watching or reading,maybe a daily affirmation could be in the learn page to motivate your learning,7,i like where it is going but obviously it is still less developed than the other pages so i did not get the full experience of it but i liked what i saw so far with including frustrations and that allowing you to learn better,completed with ease
1,AchievementsPage,user2,it is displayed my daily achievements so it helps understand or finish my goals in the frustrations i would be able to leave my feedback after an event or meet up,probably both text and options i can click on if i am in a rush based on time constraints and to help personalize,yes because i like to see achievements overcome and how i did it,yes because i can reflect on what i need to do better what were some obstacles i could improve on and what i would do different at an event next time,it shows how i can express my feelings and it should show in the learn page a new achievement i can unlock,i like the way it is right now but maybe show more trophy or reward symbols or my streak,8,i liked that i can express how i feel but it needs more motivational components like stickers or achievement symbols or something to make sure i come back to this page,completed with ease
2,AchievementsPage,user3,i see an achievements section and frustrations section i would expect this page to be about personal sustainability achievements and i would receive tips based on what my inputted frustrations are,i would expect to add new ones either in the home page or in the log page,yes because it gives me a way to check my progress,yes because i would have a place to gather all the problems to improve on for next time,compile the entries into an organized manner and show what i have learned,i think so because it would be like an encouraging message and i would feel supported,8,it would show me what i have achieved so far so that i can keep track of my progress,completed with ease
3,AchievementsPage,user4,displays my achievements and frustations of the day and a tip to make it easier to be sustainable,press on the box and write in it,the log is a good way for short term goals,i am confused what is supposed to be inputted into the frustration section,it will show me what i want to learn to make my frustrations go away,gives me badges and rankings,7,makes it clear what i want to accomplish,completed with ease
4,AchievementsPage,user5,i see a tip of the day for your sustainability goals and the achievement section so i imagine that would show the challenges you completed events you went to any rewards you made by attending events,frustrations ways to improve your habits i would think it should be autogenerated because its more motivating its more helpful to be autogenerated because its not motivating if you have to think about it yourself having to think about achievements to log would have the same effect as writing them down on paper or a journal so that would be a lot of work it is like homework versus studying if i have to manually enter my goals based on vague knowle

In [50]:
L_df

,Page Type,Name,Interaction_Options,Learn_Feature,Learn_Feature_Feedback,Progress_Feature,Quiz_Feature,Improvement_Feedback,Page_Rating,Rating_Explanation
0,LearnPage,user1,i can see what percent of completion at the top of the screen,i can see it is just a scroll up feature and there is a lot of options to click on based on categories like most popular and based on what i already know i think maybe a more bigger feed would be more clear to see,my logged habits can filter my learning which makes sense and i can also see suggestions based on what other people find helpful i can also use the search bar to look up specific topics which could be more efficient,the progress is shown above in the percent circle,i would find it helpful because i would take that as a sign that i learned enough for the day and it help me feel accomplished like i am not just doom scrolling,getting points or rewards that translate to real life rewards like coupons or vouchers for learning would be a way to motivate me just like completing real sustainable tasks i guess there could also be learn challenges that you could invite friends on,9,i really liked the learn page because it generates content based on what you entered in your log so that flow makes sense to me i also luked that youdon t have to know what to learn yet you can also go based on what other people in your level know and then as you explore you become more experienced and just keep learning from there i also think that being able to look up anything about sustainability on one app is important because first you need to adress your current needs with the world map and then you can enhance your knowledge by using the learn page so it is good for you on personal and social level
1,LearnPage,user2,i can find my percent of completion my recap of what i have learned or events i have learned from and what i can explore next,i think there should be a separate page for it but i think it is cool that you can scroll up and see the feed,you can click on the popular or more on what you know,i can see the percentage at the top,yes it would be helpful if you can get points or benefits from it not just to review,mini games like to put your goals and practice a habit like recycling games clothing games dual challenges because i like being competitive and games that unlock at an event,9,good page to have so people can learn how to recycle or other habits its helpful just to learn little things even if you are not that serious
2,LearnPage,user3,percentage what i have learned and what i can explore next,i would probably discover it on the learning page it seems like a useful layout on the learning module,i am unsure,i can track my progress through a percentage of how much i completed,yes but i think i would feel more motivated if there was some type of incentive,i think a quiz would be a good way,9,it will help me keep track of what i have learned so far and encourage me to learn more about sustainability
3,LearnPage,user4,how much i have completed what i asked the last time what i have learned so far what i can learn next,what is the exploratory learning feed,there are 3 options and it is to keep track and grow what you want to learn,by the percentage on the top corner,yes because the quiz shows up and i would want to test if i have a clear understanding of what i learned,see what your friends are learning which would motivate me to learn what others are learning,6,i think it should give me options to continue what i have learned or focus on what new things i can learn
4,LearnPage,user5,i can see the recap i think its helpful to guide your learning its cool its a continual progress you keep learning and learning i can see the percentage thing how much you finished i like that method i like that its like continued learning because it makes it feel like i will be able to keep or look back on this learning in the long term and not just after i finished a module so it makes sense to me this way,i think it is go

In [51]:
AL_df

,Page Type,Name,Page_Feedback,Interaction_Options,Search_Feature,Improvement_Feedback,Page_Preference
0,AltLearnPage,user1,i can see this is a videos page because it looks like youtube and youtube is a video content source so i have access to videos here,this is a page to learn about sustainable habits it says do at the description at the top of the page,i think you enter your search where it says watch videos on how to and the grey squares are where the video options pop up,a way to filter video content,i like the way the other page let us you learn so the idea behind that one is good but this page is executed more clearly so i guess i would prefer this page right now even if its more simplisitc but ideally merging both pages would be best
1,AltLearnPage,user2,that there is a whole page dedicated to the video feed,it is a learn page,i would use the search bar at the top,it could have options for likeing or reactions or saving videos,i like the other page but since i did say it needed it is own page for videos maybe it could use this page as just the video page
2,AltLearnPage,user3,it is pretty straightforward which i like,i can infer that it will have videos covering different topics in maintaining a sustainable lifestyle,i would search up videos based on what i want to learn for example how to recycle properly how to save energy etc,it can have a search bar and something for me to track my progress,i liked how the original learn page shows percentage completed but i like how straightforward this page is
3,AltLearnPage,user4,i like how it is simple with just videos that i can learn with,i think it will have informational videos to help me learn about sustainability,i would search it up based on the topic,it would be good to include a progress bar or percentage completed,i think it would be great to have a combined version
4,AltLearnPage,user5,i could see this is more of a scroll feed so that makes sense like how youtube already looks like,it is about learning by watching videos,there is a search bar at the top i guess that is it,i guess how long each video is like a time stamp or something similar or who posted it if its a content creator,i liked the original learn page but i guess as more things open and close its easier to navigate the learn feed if it had its own page instead of showing and hiding the feed the other way
5,AltLearnPage,user6,one thing that stood out is the availability of video tutorials,i infer this page is an instructional page for users to navigate through videos on learning how to maintain a sustainable lifestyle from the text,i would scroll through the page to find videos,i hope to see an search button where i could enter key words for specific sustainable goals to learn about,i prefer this page more than the original one because i think videos are easier to understand
6,AltLearnPage,user7,the video feature stands out to me,it shows your profile and learning,i would like a search bar to find a specific video or click on the suggested videos,watching videos should not be the main page unless it is a video based platform main page should be like a profile or daily affirmations vlogs goals,i prefer the original learn page i would like to see videos in the final learn page
